In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import copy


In [ ]:
def sort_array(array):
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return ranks
def matrix2vec(matrix):
    d = len(matrix)
    interactions = np.zeros([int(d * (d-1) / 2),1])
    k = 0
    for i in range(d):
        for j in range(i+1, d):
            interactions[k] = matrix[i][j]
            k = k + 1  
    return interactions
def matric2dic(hessian, K):
    IS = {}
    for i in range(len(hessian[0])):
        for j in range(i+1, len(hessian[0])):
            tmp = 0
            interation = 'Interaction: '
            interation = interation + str(i + 1) + ' ' + str(j + 1) + ' '
            IS[interation] = hessian[i][j]
    Sorted_IS = [(k, IS[k]) for k in sorted(IS, key=IS.get, reverse=True)]
    return IS, Sorted_IS
def vec2dic(vector, num_dim,len_gt):
    IS = {}
    tmp = 0
    for i in range(num_dim):
        for j in range(i+1, num_dim):
            interation = str(i + 1) + ',' + str(j + 1) 
            IS[interation] = vector[tmp]
            tmp = tmp + 1
    Sorted_IS = dict(sorted(IS.items(), key=lambda item: item[1],reverse=True)[:len_gt])

    dictlist=[]
    for key, value in Sorted_IS.items():
        temp = ((int(key.split(",")[0]),int(key.split(",")[1])),value)
        dictlist.append(temp)
    return dictlist

## 1. Load the data (Energy efficiency Data Set from UCI)
Data is from: A. Tsanas, A. Xifara: 'Accurate quantitative estimation of energy performance of residential buildings using statistical machine learning tools', Energy and Buildings, Vol. 49, pp. 560-567, 2012

In [ ]:
# 1. Data preprocessing
def data_generating_energy(traindf, testdf):

    trainx = traindf.drop('Y2', axis=1).values; trainy = traindf['Y2'].values
    testx = testdf.drop('Y2', axis=1).values; testy = testdf['Y2'].values

    scalerx = preprocessing.MinMaxScaler().fit(trainx)
    trainx = scalerx.transform(trainx)+ 0.5; testx = scalerx.transform(testx)+ 0.5

    scalery = preprocessing.MinMaxScaler().fit(trainy.reshape(-1,1))
    trainy = scalery.transform(trainy.reshape(-1,1)).reshape(1, -1); testy = scalery.transform(testy.reshape(-1,1)).reshape(1, -1)

    # transfer data into tensor
    print(trainy,trainx)
    x = torch.tensor(trainx, dtype = torch.float); y = torch.tensor(trainy[0], dtype = torch.float)
    x_test = torch.tensor(testx, dtype = torch.float); y_test = torch.tensor(testy[0], dtype = torch.float)
    
    return x, y, x_test, y_test

## 2. Interaction effect measure

### 2.0 Group Expected Hessian

In [ ]:
# GEH
def inputHessian_dropout(mlp, r):
    Hessian = []
    output = mlp(r)
    first = torch.autograd.grad(output, r, create_graph=True)
    for i in range(len(r[0])):
        gradient = torch.zeros(len(r[0]), dtype = torch.float)
        gradient[i] = 1.0
        second = torch.autograd.grad(first, r, grad_outputs=gradient.view(1,-1), retain_graph=True)
        Hessian.append(second[0][0].tolist())
    return Hessian
def AEH(mlp, num_dim, data):
    Hessian = np.zeros([num_dim, num_dim])
    n_data = data.shape[0]
    for i in range(n_data):
        r = data[i].clone().detach().requires_grad_(True).view(1,-1)
        Hessian = Hessian + np.array(inputHessian_dropout(mlp, r))
    Hessian = abs(Hessian) / n_data 
    HS, sorted_HS = matric2dic(Hessian, 10)
    return sorted_HS, Hessian

## Group Expected Hessian
def GEH(mlp, data, num_cluster,feature_size):
    kmeans = KMeans(n_clusters=num_cluster, random_state=0).fit(data)
    data_sep = []
    data_number = 0
    for i in range(num_cluster):
        data_i = data[(kmeans.labels_ == i).tolist()]; data_i = torch.tensor(data_i, dtype=torch.float)
        data_sep.append(data_i)
        data_number = data_number + data_i.shape[0]
    mlp.eval()
    Hessian = np.zeros([num_feature, num_feature])
    for i in range(num_cluster):
        _, hessian = AEH(mlp, num_feature, data = data_sep[i])
        Hessian = Hessian + hessian * data_sep[i].shape[0] / data_number  
    return Hessian

### 2.1 Bayesian Group Expected Hessian

In [ ]:
# Bayesian GEH
def MC_inputHessian(mlp, r, fixed_noise):
    Hessian = []
    output = mlp(r, fixed_noise)
    first = torch.autograd.grad(output, r, create_graph=True)
    for i in range(len(r[0])):
        gradient = torch.zeros(len(r[0]), dtype = torch.float)
        gradient[i] = 1.0
        second = torch.autograd.grad(first, r, grad_outputs=gradient.view(1,-1), retain_graph=True)
        Hessian.append(second[0][0].tolist())
    return Hessian

def MC_AEH(mlp, num_dim, fixed_noise, data):
    Hessian = np.zeros([num_dim, num_dim])
    n_data = data.shape[0]
    for i in range(n_data):    
        r = data[i].clone().detach().requires_grad_(True).view(1,-1)
        Hessian = Hessian + np.array(MC_inputHessian(mlp, r, fixed_noise))
    Hessian = abs(Hessian) / n_data
    HS, sorted_HS = matric2dic(Hessian, num_dim)
    return sorted_HS, Hessian

def MC_GEH_cluster(mlp, data_sep, num_cluster,feature_size):
           
    mlp.train()
    fixed_noise = [torch.rand(feature_size),torch.rand(140),torch.rand(100),torch.rand(60),
                   torch.rand(20), torch.rand(10)]
    Hessian = np.zeros([num_feature, num_feature])
    data_number = 0
    for i in range(num_cluster):
        data_number = data_number + data_sep[i].shape[0]
        
    for i in range(num_cluster):
        _, hessian = MC_AEH(mlp, num_feature, fixed_noise, data = data_sep[i])
        Hessian = Hessian + hessian * data_sep[i].shape[0] / data_number
        
    return Hessian

### 2.2 Determine the optimal number of clusters

In [ ]:
def spearman_rank(Hessian0, Hessian1):
    length = Hessian0.shape[0]
    lti = np.tril_indices(length, -1)
    attribution0 = Hessian0[lti] / np.sum(Hessian0[lti]); attribution1 = Hessian1[lti] / np.sum(Hessian1[lti])
    rank0 = sort_array(attribution0); rank1 = sort_array(attribution1)
    return np.sum(((attribution0-attribution1) ** 2 ) * ((rank0 - rank1) ** 2))

In [ ]:
def delta_M(mlp, data, NUM_Cluster = 60):
    increase = []
    Hessian0 = GEH(mlp, data, 1)
    for i in range(NUM_Cluster - 1):
        num_cluster = i + 2
        hessian = GEH(mlp, data, num_cluster)
        increase.append(spearman_rank(Hessian0,hessian))
        Hessian0 = hessian
    return np.array(increase)

## 3. Model Interaction via Concrete Dropout
Gal, Yarin, Jiri Hron, and Alex Kendall. "Concrete dropout." Advances in neural information processing systems. 2017.

In [ ]:
#### We add a main effect to improve the NN training, which could be omited as well.
class Main_effect(nn.Module):
    def __init__(self, num_dim):
        super(Main_effect, self).__init__()
        self.fc1 = nn.Linear(num_dim, 1)
    def forward(self, x):
        x = self.fc1(x)
        return x

Different from the original concrete dropout, we control the random mask of each node manually by providing the noise. This is used in Bayesian GEH.

In [ ]:
# Concrete Dropout Layer
class ConcreteDropout(nn.Module):
    def __init__(self, size = 1, weight_regularizer=1e-6,
                 dropout_regularizer=1e-5, init_min=0.1, init_max=0.1):
        
        super(ConcreteDropout, self).__init__()
        
        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        
        init_min = np.log(init_min) - np.log(1. - init_min)
        init_max = np.log(init_max) - np.log(1. - init_max)
        
        self.p_logit = nn.Parameter(torch.empty(size).uniform_(init_min, init_max))

    def forward(self, x, layer, noise):
        p = torch.sigmoid(self.p_logit)
            
        out = layer(self._concrete_dropout(x, p, noise))

        sum_of_square = 0
        
        network_weights = torch.sum(torch.sum(torch.pow(layer.weight, 2), 0) / (1 - p))
        network_bias = torch.sum(torch.pow(layer.bias, 2))   
            
        weights_regularizer = self.weight_regularizer * (network_bias + network_weights)

        dropout_regularizer = p * torch.log(p) + (1. - p) * torch.log(1. - p)
        dropout_regularizer = self.dropout_regularizer * torch.sum(dropout_regularizer)

        regularization = weights_regularizer + dropout_regularizer

        return out, regularization
        
    def _concrete_dropout(self, x, p, noise):
        eps = 1e-7
        temp = 0.1
        
        if type(noise) == int: ## shape of unif_noise is [num_data, dim]
            unif_noise = torch.rand_like(x)
        else:
            unif_noise = noise
            
        drop_prob = (torch.log(p + eps)
                    - torch.log(1 - p + eps)
                    + torch.log(unif_noise + eps)
                    - torch.log(1 - unif_noise + eps))
        
        drop_prob = torch.sigmoid(drop_prob / temp)
        random_tensor = 1 - drop_prob
        retain_prob = 1 - p
        
        x  = torch.mul(x, random_tensor)
        x /= retain_prob
        
        return x

In [ ]:
# MLP with concrete dropout layer
class Model_NC_softplus(nn.Module):
    def __init__(self, weight_regularizer, dropout_regularizer,input_dimension):
        super(Model_NC_softplus, self).__init__()
        self.linear1 = nn.Linear(input_dimension, 140)
        self.linear2 = nn.Linear(140, 100)
        self.linear3 = nn.Linear(100, 60)
        self.linear4 = nn.Linear(60, 20)
        self.linear5 = nn.Linear(20, 10)
        self.linear6 = nn.Linear(10, 1)

        self.conc_drop1 = ConcreteDropout(size = input_dimension,  weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop2 = ConcreteDropout(size = 140, weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop3 = ConcreteDropout(size = 100, weight_regularizer=weight_regularizer,
                                             dropout_regularizer=dropout_regularizer)    
        self.conc_drop4 = ConcreteDropout(size = 60,  weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop5 = ConcreteDropout(size = 20, weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop6 = ConcreteDropout(size = 10, weight_regularizer=weight_regularizer,
                                             dropout_regularizer=dropout_regularizer)    
    def forward(self, x, NOISE = [0,0,0,0,0,0]): ## if the noise is not given, then generate a standard Gaussian noise
        softplus = nn.Softplus(beta = 10)
        if self.training:
            regularization = torch.empty(6)

            x1, regularization[0] = self.conc_drop1(x, self.linear1, NOISE[0])
            x1 = softplus(x1)
            x2, regularization[1] = self.conc_drop2(x1, self.linear2, NOISE[1])
            x2 = softplus(x2)
            x3, regularization[2] = self.conc_drop3(x2, self.linear3, NOISE[2])
            x3 = softplus(x3)
            x4, regularization[3] = self.conc_drop4(x3, self.linear4, NOISE[3])
            x4 = softplus(x4)
            x5, regularization[4] = self.conc_drop5(x4, self.linear5, NOISE[4])
            x5 = softplus(x5)
            output, regularization[5] = self.conc_drop6(x5, self.linear6, NOISE[5])
            return output, regularization.sum()
        else:
            x = softplus(self.linear1(x))
            x = softplus(self.linear2(x))
            x = softplus(self.linear3(x))
            x = softplus(self.linear4(x))
            x = softplus(self.linear5(x))
            x = self.linear6(x)            
            return x
def heteroscedastic_loss(true, output):
    return torch.mean(torch.sum((true - output)**2, 1), 0)

## 4. Experiments

### 4.0 Train a Concrete dropout NN
About the trainining a Concrete dropout NN, please refer to [Gal et al., 2017] https://github.com/yaringal/ConcreteDropout

In [ ]:
def get_anyorder_R_precision(interactions, ground_truth):

    R = len(ground_truth)
    recovered_gt = []
    counter = 0

    for inter, strength in interactions:

        inter_set = set(inter)  # assume 1-indexed
        #print(inter_set)
        #print(ground_truth)
        if any(inter_set <= gt for gt in ground_truth):
            #print("Good")
            
            counter += 1
    R_precision = counter / R

    return R_precision

def imuncertainty(mlp, data, num_cluster, n_iter = 20):

    kmeans = KMeans(n_clusters=num_cluster, random_state=0).fit(data)
    data_sep = []
    for i in range(num_cluster):
        data_i = data[(kmeans.labels_ == i).tolist()]; data_i = torch.tensor(data_i, dtype=torch.float)
        data_sep.append(data_i) 

    print('############# The 1st interation for uncertainty calculation')
    # Initialize the first interaction
    Hessian = MC_GEH_cluster(mlp, data_sep, num_cluster,data.shape[1])
    int_unc = matrix2vec(Hessian)

    # Rest of Interaction
    for num_int in range(n_iter - 1):
        if ((num_int + 2) % 10 == 0):
            print('############# The %dth interation for uncertainty calculation' %(num_int + 2))
        Hessian = MC_GEH_cluster(mlp, data_sep, num_cluster,data.shape[1])
        int_unc_new = matrix2vec(Hessian)
    
        int_unc = np.concatenate((int_unc, int_unc_new), axis=1)

    mean_interaction = np.mean(int_unc, axis = 1)
    std_interaction = np.std(int_unc, axis = 1)
    #print('mean of each interaction is')
    #print(mean_interaction)
    #print('std of each interaction is')
    #print(std_interaction)
    
    return mean_interaction, std_interaction

class EarlyStopping:
    def __init__(self, tolerance, patience):
        self.tolerance = tolerance
        self.patience = patience
    def stop_criterion(self, val_errors):
        if len(val_errors) < self.patience + 1:
            return False
        else:
            current_best = min(val_errors[:-self.patience])
            current_stop = True
            for i in range(self.patience):
                current_stop = current_stop and (val_errors[-i-1] - current_best > self.tolerance)
            return current_stop
def training_CD_soft(mlp, main_effect, x, y,x_val,y_val, x_test, y_test,gt, learning_rate = 0.001, anneling = 1000, batch_size = 50, num_epoch=150, tolerance=0.002, patience = 20):
    parameters = set(main_effect.parameters()) | set(mlp.parameters())
    optimizer = optim.Adam(parameters, lr = learning_rate)
    early_stop = EarlyStopping(tolerance, patience)
    criterion = nn.MSELoss()

    train_errors = []
    val_errors = []

    num_data, num_dim = x.shape
    y = y.view(-1, 1)
    data = torch.cat((x, y), 1)
    results=[]
    annel_index = 0
    best_loss=999999
    for epoch in range(num_epoch):
        # permuate the data
        data_perm = data[torch.randperm(len(data))]
        x = data_perm[:, 0:-1]
        y = data_perm[:, -1]

        for index in range(int(num_data/batch_size)):
            # data comes in
            inputs = x[index*batch_size : (index+1)*batch_size]
            labels = y[index*batch_size : (index+1)*batch_size].view(-1,1)

            # initialize the gradient of optimizer
            optimizer.zero_grad()
            mlp.train()
            # calculate the loss function
            output_mlp, reg = mlp(inputs)
           # loss with var   
#             output_mlp, var, reg = mlp(inputs)          
#             loss = heteroscedastic_loss(labels, output_mlp + main_effect(inputs), var) + reg

            # calculate the loss function
            coef_annel = min(1, 0.01 + annel_index / anneling)
        
            loss = heteroscedastic_loss(labels, output_mlp + main_effect(inputs)) + coef_annel * reg
            # backpropogate the gradient     
            loss.backward()
            # optimize with SGD
            optimizer.step()
            
            annel_index += 1
        # train and validation loss
       


        if (epoch % 10) == 0:
            mlp.eval()
            train_errors.append(criterion(mlp.forward(x) + main_effect.forward(x), y.view(-1,1)))
            val_errors.append(criterion(mlp.forward(x_val) + main_effect.forward(x_val), y_val.view(-1,1)))
            print('EPOCH %d: TRAIN LOSS: %.4f; VAL LOSS IS: %.5f.'% (epoch+1, train_errors[-1], val_errors[-1]))
            if val_errors[-1] < best_loss:
                best_loss = val_errors[-1]
                best_net = copy.deepcopy(mlp)
    print("Test:",criterion(best_net.forward(x_test) + main_effect.forward(x_test), y_test.view(-1,1)))
    mean_interaction, std_interaction = imuncertainty(best_net, x_test.numpy(),  x_train.shape[1])
    Sorted_IS = vec2dic(mean_interaction, x_train.shape[1],len(gt))
    R=get_anyorder_R_precision(Sorted_IS,gt)
    print(R,Sorted_IS)
    return R

In [ ]:
x_test=np.genfromtxt("../x_testF1.csv")
y_test=np.genfromtxt("../y_testF1.csv")
x_test = torch.tensor(x_test, dtype = torch.float)
y_test = torch.tensor(y_test, dtype = torch.float)
num_data=x_train.shape[0]
num_feature=x_train.shape[1]
gt=[{1, 2},{1,3},{2,3}, {3, 5}, {9, 10}, {8, 9},{8,10},{7,9},{7,10},{7, 8},{2,7}]
results=[]
wr = l**2. /num_data
# training parameters:
learning_rate = 0.01;
l = 1e-7
dr = 0.01 * 2 / num_data 
batch_size = 32; num_epoch = 150;
tolerance = 0.01; patience = 20;
anneling = 10 * num_data / batch_size
for i in range(10):
    x_train=np.genfromtxt("../x_trainF1"+str(i)+".csv")
    y_train=np.genfromtxt("../y_trainF1"+str(i)+".csv")
    x_val=np.genfromtxt("../x_valF1"+str(i)+".csv")
    y_val=np.genfromtxt("../y_valF1"+str(i)+".csv")
    x_train = torch.tensor(x_train, dtype = torch.float) 
    y_train = torch.tensor(y_train, dtype = torch.float)
    x_val = torch.tensor(x_val, dtype = torch.float) 
    y_val = torch.tensor(y_val, dtype = torch.float)
    mlp = Model_NC_softplus(wr, dr,x_train.shape[1])
    main_effect = Main_effect(num_feature)
    R=training_CD_soft(mlp, main_effect, x_train, y_train,x_val,y_val, x_test, y_test,gt, learning_rate, anneling, batch_size, 150, tolerance, patience)
    results.append(R)

In [ ]:
mean=np.mean(results)
sd=np.std(results)
print(mean,sd)